In [1]:
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
from rnn_cell import PredictTime
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: ", device)

df = get_data()

Using device:  cuda


In [2]:

def make_data(df, device):

    x_train, y_train = [], []
    prev = None

    for row in df:
        if prev is None:
            prev = row
        x_train.append(prev)
        y_train.append(row)
        prev = row
    print(x_train[0].size())
    return torch.stack(x_train).float().to(device),torch.stack(y_train).float().to(device)

make_data(get_binary_values(get_data()), device)



torch.Size([1])


(tensor([[1.],
         [1.],
         [0.],
         ...,
         [1.],
         [0.],
         [1.]], device='cuda:0'),
 tensor([[1.],
         [0.],
         [1.],
         ...,
         [0.],
         [1.],
         [1.]], device='cuda:0'))

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from IPython.display import clear_output
import torch.nn as nn
import random

batch_size = 200
x_d, y_d = make_data_scalar(get_data(), device)

print(y_d.size(), x_d.size())

model = PredictTime(input_size=x_d[0].size()[0],
                    output_size=y_d[0].size()[0],
                    hidden_layers=5,
                    hidden_size=100, device=device).to(device)
optimizer = optim.Adam(model.parameters())

loss = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(x_d,y_d), batch_size=batch_size)
epochs = 5000
for e in range(epochs):
    model.train()
    #print(next(iter(loader)))
    model.clean_state()
    res = []
    
    
    if random.random() < e/(2*epochs):
        model.teacher_forcing = False
    else:
        model.teacher_forcing = True
    
    for x, y in loader:
        
        if random.random() < 0.3:
            continue
        model.init_state()
        
        
        y_pred = model(x)
        l = loss(y_pred, y)
        res.append(l)
        #print(y_pred, y)
    
    
    l = res[0]
    for i in res[1:]:
        l += i
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if e % 10 != 0:
        continue
    #clear_output(wait=True)
    sum_loss = 0
    #print(list(model.parameters())[-1])

    for x, y in loader:
        model.eval()
        with torch.no_grad():
            y_pred = model(x)
            sum_loss += np.sqrt(loss(y_pred, y).cpu())
    
    print("Epoch %d Loss %.4f" % (e, sum_loss))
    
    
#for d in df.values:

Max value:  1000
torch.Size([8152, 1]) torch.Size([8152, 1])
Epoch 0 Loss 16.5944
Epoch 10 Loss 14.8649
Epoch 20 Loss 14.8731
Epoch 30 Loss 14.7688
Epoch 40 Loss 14.7558
Epoch 50 Loss 14.7164
Epoch 60 Loss 14.6901
Epoch 70 Loss 14.5990
Epoch 80 Loss 14.5690
Epoch 90 Loss 14.5044
Epoch 100 Loss 14.4620
Epoch 110 Loss 14.4385
Epoch 120 Loss 14.4359
Epoch 130 Loss 14.3682
Epoch 140 Loss 14.3404
Epoch 150 Loss 14.3133
Epoch 160 Loss 14.3321
Epoch 170 Loss 14.2625
Epoch 180 Loss 14.2787
Epoch 190 Loss 14.2563
Epoch 200 Loss 14.2144
Epoch 210 Loss 14.2077
Epoch 220 Loss 14.1991
Epoch 230 Loss 14.2095
Epoch 240 Loss 14.1927
Epoch 250 Loss 14.1858
Epoch 260 Loss 14.1869
Epoch 270 Loss 14.1820
Epoch 280 Loss 14.1846
Epoch 290 Loss 14.1798
Epoch 300 Loss 14.1720
Epoch 310 Loss 14.9659
Epoch 320 Loss 14.4288
Epoch 330 Loss 14.2390
Epoch 340 Loss 14.2212
Epoch 350 Loss 14.1966
Epoch 360 Loss 14.1961
Epoch 370 Loss 14.1824
Epoch 380 Loss 14.4109
Epoch 390 Loss 14.2812
Epoch 400 Loss 14.8325
Epoch 4

In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    
    fig, ax = plt.subplots(2)
    
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result when feeding correct values as input, mixed teacher forcing and not forcing")
    fig.savefig("mixed_teacher_forcing.png")
    
model.teacher_forcing = True

amount = 1000
eval_model(x_d,y_d,model)


In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        prev = val
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    
    fig, ax = plt.subplots(2)
    
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result without feeding correct values as input, mixed teacher forcing and not forcing")
    fig.savefig("mixed_teacher_forcing_not_correct.png")

model.teacher_forcing = True
amount = 1000
eval_model(x_d,y_d,model)
